# Demystifying Neural Network

### Improving neural network's learning : cost function and activation function
- Intro : Don't Forget that Neural Network is also a Function!
- problem with the combination of the mean squared error cost function and the sigmoid function
- the combination of softmax and the cross-entropy cost function
    - softmax
    - cross-entropy
    - How does the cross entropy loss function solve the learning slowdown problem?
        - derivation of partial derivatives
- The difference!

### Intro : Don't Forget that Neural Network is also a Function!

Last week, the instructor of my NLP (Natural Language Processing) basics with Pytorch class mentioned that we should not forget that the objective of machine learning is generalization: to have a good prediction for unseen data. In the midst of an A.I. revolution, we tend to think that neural network is just the 'cure-for-all-problems' and forget that it is also a function that approximates a true distribution based on sampled data. His lecture helped me to motivate myself to review some statistics materials that I have abandoned for few weeks. I especially studied the relationship between maximum likelihood estimation and the cross entropy loss, which will be reflected on this notebook. One of the excellent posts that I highly recommend and helped me a lot is the post on cross-entropy loss by Rob DiPietro.

In this notebook and upcoming ones, I am going to write about how to improve neural network's learning. There are several different techniques to improve its performance, which I will get to shortly with upcoming notebooks. Today, I am going to focus on a better choice of cost function, the cross-entropy cost function, and another choice of activation function, the softmax function. Both of them are readily used for classification (and it makes sense to only use it for classification). These two functions help to prevent our precious gradients from vanishing during backpropagation. Gradients are the key ingradeint for a neural network to learn, but why on earth would a gradient vanish? 

### problem with the combination of the mean squared error cost function and the sigmoid function

We know that neural network learns via backpropagation. In part 2, I used the mean squared error cost function and the sigmoid activation function. Our cost function was: $C = \dfrac{1}{2}(y-a)^2$. Here, $y$ denotes desired output. $a$ represents the signomid activation function, $a= \sigma(z)$, where $z=wx+b$. The sigmoid function gives us predicted value. To understand better, let's say that our our input $x=1$. The partial derivatives of the cost function are:   
$$\dfrac{\partial C}{\partial w} = (a-y) \sigma'(z)x$$  
$$\dfrac{\partial C}{\partial b} = (a-y) \sigma'(z)$$

Think about a case when our desired output $y=1$ but the predicted value was 0.0000001. Our error is significantly big, right? Intuitively, it would make sense for our implemented neural network to learn better since the error is big. Would that be the case? Let's look at the shape of the sigmoid activation function.

<img src="img/blog4_figure1.png" width="300" height="200" />

As we can see, when our predicted output is close to either 0 or 1, the curve gets very flat. That is, our $\sigma'(z)$ gets very small, almost 0. Since we multiply $(a - y)$ with the derivative of the sigmoid function $\sigma'(z)$ to get the partial derivatives, $\dfrac{\partial C}{\partial w}$ and $\dfrac{\partial C}{\partial b}$ also get very small. This causes our neural network to slow down, since there is no 'learning'; our gradient is near 0. We can get luckly and gradients might survive for 1 or 2 layers, but if we have a deep neural network, gradients will not be able to propagate to the bottom layer (layer close to the input layer). 

However, it is important to note that for regression, the mean squared error cost function performs fine. It becomes problem in classification, as we use the sigmoid activation function to make our predicted value look like a probability. 

So here are 2 major drawbacks with the combination of the squared error cost function and the sigmoid activation function: 
- If an error is big, there is almost no gradient for a sigmoid unit to fix up the error
- If we are trying to assign probabilities to mutually exclusive class labels, we know that the output should sum to 1. However, with the sigmoid activation function, we are depriving the network of this knowledge

So, do we have a different cost function that works better than the combination of the mean squared error function and the sigmoidfunction? **Yes! There is!** It's the combination of the **cross-entropy cost function** and the **softmax function**. This new combination solves the drawbacks listed above. It forces the outputs to represent a probability distribution across discrete alternatives.  

### the combination of softmax and the cross-entropy cost function
#### Softmax 
The name softmax comes from the fact that it's the opposite concept of 'hardmax function' such as, $max(0,1)=1.$ It's a soft continuous version of the maximum function. This softmax function is applied to the output layer, instead of the sigmoid function. Below is the formula of the softmax function:
$$a_i = \dfrac{e^{z_i}}{\sum_k e^{z_k}}$$
The formula computes the exponential of the given output value $z_i$ from the previous layer (second to the last) and divides it by the sum of exponential values of all $k$ number of the outputs $z_k$. This ensures that the sum over all outputs remains 1. Moreover, above formula implies that the output activations are all positive, since the exponential of real number is always positive. Therefore, Unlike the sigmoid function, the softmax function gives us the nice interpretation of outputs of a neural network, since the output from the softmax layer can be thought of as a probability distribution.

#### cross-entropy

The cross-entropy cost function is the right cost function to use with the softmax function. Let's step back for a moment. When we develop a model for probabilistic classification, we aim to map the inputs of the model to probabilistic predictions. Then, we train our model by adjusting the model's weights and biases so that our approximations get closer to ground truth probabilities. Let's say there are two labels for our data: $[cat, dog]$. We pass an image of a dog and the output of our model is $[P(cat | data), P(dog | data)] = [0.4, 0.6]$. We want to make sure this gets close to $[0, 1]$, 100% certainty for dog and 0 for cat. It turns out that cross-entropy is a better measure than mean squared error to measure the difference between $predicted$ and $ground truth$. 

Let's first talk about important concepts, entropy, cross-entropy, and KL divergence, to get to the corss-entropy cost function. 
**Entropy** is the expected number of bits under optimal encoding. The formula of entropy is: 
$$H(y) = -\sum y_i log \frac{1}{y_i}$$ 
That is, entropy computes the number of bits we will need if we use the correct tool, the ground truth probability of a given label ($y_i$). On the other hand, **cross entropy**, is the number of bits we will need if we encode symbols from y using the wrong tool (in our case neural network's estimate of the probability that the correct output is $i$, $\hat{y_i}$). Therefore, the formula of cross entropy takes into account of the fact that we are using the wrong tool to encode a given symbol. 
$$ H(y, \hat{y_i}) = \sum_i y_i log \frac{1}{\hat{y_i}} = - \sum_i y_i log {\hat{y_i}}$$
Since entropy represents the optimal number of bits, cross entropy is always larger than entropy.

**KL divergence** from $\hat{y}$ to $y$ is simply the distance between cross entropy and entropy. KL divergence can be written as:
$$KL(y || \hat{y_i}) = \sum_i y_i log \frac{1}{\hat{y_i}} - \sum_i y_i log \frac{1}{y_i} =  \sum_i y_i log \frac{y_i}{\hat{y_i}}$$
We can think of the value of KL divergence as the number of extra bits needed on average to achieve optimal encoding using $y_i$. This value is never negative (because of the log), and it becomes 0 when $y_i$ and $\hat{y_i}$ are the same. As we can see from the formulas above, minimizing cross entropy is same as minimizing KL divergence since $y$ is the ground truth value (a constant) and does not depend on $y_i$. That is, we can use either cross entropy or KL divergence as a loss function and find the optimal parameters via gradient descent. Moreover, our ground truth value $y_i$ is always 1 (probability of 1) for classification problem when using softmax which is nice. Thus, the cross-entropy cost function becomes negative log likelihood $ C= - \sum_i log \hat{y_i} $.

As I mentioned in the intro section, I would like to mention briefly about the relationship between cross entropy loss minimization with maximum likelihood estimation. By minimizing cross entropy loss function, a neural network can approximate a real distribution as close as possible. Furthermore, we can also regard our loss function as a direct measure of a model's predictive power with respect to training data. How?

Let's look at our cost function again, $ C= - \sum_i y_i log \hat{y_i} $. When our learning algorithm is trying to make this loss small, the only way to make that small is make $-log\hat{y_i}$ small, which is to make $\hat{y_i}$ big (note that since $\hat{y_i}$ is a probability, this can never be bigger than 1). What our cost function does is, it looks at the ground truth label of our training data and tries to make the corresponding probablity of that label as high as possible. This takes the form of maximum liklihood estimation. That is, our cost function adjusts our parameters so that the likelihood of our data under the model is maximized. 

Therefore, On one hand, minimizing cross entropy lets us get closer to ground truth distribution with our model. On the other hand, minimizing cross entropy is same as minimizing the negative log likelihood of our data.

#### How does the cross entropy cost function solve the learning slowdown problem?
We went through all the concepts of softmax, cross entropy, and KL divergence to understand that the combination of the softmax function and the cross entropy cost function is better for our model's learning. Recall that a gradient of the sigmoid activation function are very small value when an predicted output is close to either 0 or 1? Let's see what would be the partial derivatives of the cross entropy cost function with the softmax function. 

Let's derive the partial derivatives of softmax:

<img src="img/blog4_figure2.png" width="600" height="300" />

Then, the derivative of the cross entropy cost function with respect to softmax layer:

<img src="img/blog4_figure3.png" width="500" height="300" />

<img src="img/blog4_figure4.png" width="500" height="300" />

### The difference!
Can you see the difference between the partial derivatives of the mean squared error cost function and those of the cross-entropy cost function? The partial derivatives of the cross entropy cost function does not have the derivative of the sigmoid function. Therefore, it is less likely for gradients of the cross entropy cost function to vanish or get very small. Well, if the input $a_k$ is near to 0, the corresponding weight $w_k$ will learn slowly. However, still, it's better than training networks with the mean squared error cost function and the sigmoid function for classification problem. I remember listening to a lecture on gradients vanishing (I think it was CS 231n), in which a lecturer mentions that it is best to avoid using the sigmoid activation function due to gradient vanishing problem.

Moreover, the cross entropy cost function has a very big gradient when the target value is 1 and the output is almost 0, this is when our network's predicted value is very wrong. Recall that with the sigmoid function, when our network is very wrong at approximating value, gradient has a lot of difficulty learning. Thus, with the combination of softmax and the cross entropy cost function, we can also think intuitively through our gradient's value: 'more wrong' means big gradient, 'less wrong' or 'almost right' means small gradient. 

For the next notebook, I will be focusing on regularization of neural network.